# Episodio 3 - Simulaciones clásicas y control de calidad

Minimización y simulación corta con OpenMM.

In [ ]:
from openmm import app
import openmm as mm
from openmm import unit

pdb = app.PDBFile('../../data/alanine-dipeptide.pdb')
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(300 * unit.kelvin, 1.0 / unit.picosecond, 2.0 * unit.femtoseconds)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

simulation.minimizeEnergy(maxIterations=100)
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(200)

state = simulation.context.getState(getEnergy=True)
print('Potential energy:', state.getPotentialEnergy())